In [ ]:
!pip install --quiet scvi-colab
from scvi_colab import install
install()


import rich

import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install --quiet scrublet
    import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import anndata
import scvi
import scanpy as sc

sc.set_figure_params(figsize=(4, 4))
scvi.settings.seed = 94705

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

!pip install anndata2ri
import anndata2ri

!pip install session_info

seed=10
sc.logging.print_versions()

INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


/usr/local/lib/python3.10/dist-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/usr/local/lib/python3.10/dist-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done


INFO: Global seed set to 94705
INFO:lightning.fabric.utilities.seed:Global seed set to 94705


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.3/219.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rpy2: filename=rpy2-3.5.14-cp310-cp310-linux_x86_64.whl size=329837 sha256=2c4b81c2c1c804a02c00b091db2203a5abfbab76938c5ecc5ba7fd0130803ae8
  Stored in directory: /root/.cache/pip/wheels/d4/b5/2f/529c5de71addd73d8e3c51104721d9e084c97ed2ce1c69de2d
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2
-----
anndata     0.10.3
scanpy      1.9.6
-----
PIL                 9.4.0
absl                NA
aiohttp             3.9.1
aiosignal           1.3.1
anndata2ri          1.3.1
anyio               NA
astunparse          1.6.3
async_timeout       4.0.3
attr                23.1.0
backcall            0.2.0
backoff             2.2.1
bs4               

In [ ]:
#load reference adata
adata = scvi.data.read_h5ad(".../Atlas_human_extension_II_3000HVG_cleaned2.h5ad")


In [ ]:
model = scvi.model.SCVI.load("/path/to/model/", adata=adata, use_gpu=True)
model


In [ ]:
adata

In [ ]:
seed = 10

In [ ]:
adata_path = "/content/drive/MyDrive/Konstantin/Atlas_human_extension_II_3000HVG_cleaned2.h5ad"
model_path = "/content/drive/MyDrive/Konstantin/models_graz/scanvi_2"

In [ ]:
model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 30, n_layers: 3, dropout_rate: 0.1, dispersion: gene, gene_likelihood: nb, 
latent_distribution: normal
Training status: Trained
Model's adata is minified?: False

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(model, labels_key="annotation_final_level1B", unlabeled_category = "Unknown")

In [ ]:



scanvi_model.train(max_epochs = 100, plan_kwargs={"lr":0.001}, early_stopping = True, early_stopping_patience = 15)




INFO     Training for 100 epochs.                                                                                  


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 100/100: 100%|██████████| 100/100 [4:15:32<00:00, 152.80s/it, v_num=1, train_loss_step=1.11e+3, train_loss_epoch=1.04e+3]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [4:15:32<00:00, 153.32s/it, v_num=1, train_loss_step=1.11e+3, train_loss_epoch=1.04e+3]


In [ ]:
scanvi_model.save(model_path)

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scANVI")
sc.tl.umap(adata)

In [ ]:
adata.write(adata_path)
